##  Préparation du jeu de données anglais–français

Nous utilisons le dataset public de Kaggle Language Translation (English–French) contenant deux colonnes : phrases anglaises et leurs équivalents français.  
Les données sont chargées, renommées, nettoyées (valeurs manquantes, doublons, longueurs extrêmes) puis divisées en deux sous-ensembles :  
-Train : 90 % des exemples pour l’entraînement du modèle  
-Validation : 10 % pour l’évaluation des performances  

Après nettoyage, on obtient environ 158 000 paires pour l’entraînement et 17 000 pour la validation.  
Ce dataset servira à alimenter le modèle de traduction basé sur l’architecture BERT (encodeur) et GPT-2 (décodeur).


In [121]:
import pandas as pd

In [37]:
df=pd.read_csv("eng_-french.csv")


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = df.rename(columns={
    "English words/sentences": "en",
    "French words/sentences": "fr"
})

df = df.dropna()
df = df.drop_duplicates()
df = df[(df["en"].str.len() > 2) & (df["fr"].str.len() > 2)]
df = df[df["en"].str.len() < 200]
df = df[df["fr"].str.len() < 200]

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(f"Train: {len(train_df)} | Val: {len(val_df)}")


Train: 158051 | Val: 17562


##  Initialisation des modèles et tokenizers

Dans cette partie, on importe les bibliothèques nécessaires (**Transformers**, **PyTorch**) et on charge deux modèles préentraînés :

- **Encodeur : `bert-base-uncased`**  
  Sert à comprendre les phrases anglaises et à produire leurs représentations vectorielles (embeddings contextuels).

- **Décodeur : `dbddv01/gpt2-french-small`**  
  Sert à générer les phrases traduites en français à partir des représentations fournies par l’encodeur.

Les **tokenizers** correspondants sont également chargés pour convertir le texte brut en tokens numériques compatibles avec chaque modèle.  
L’objectif est de créer une architecture **encoder–decoder hybride** :  
BERT encode la phrase anglaise → GPT-2 décode et génère la traduction française.


In [33]:
from transformers import AutoTokenizer, AutoModel, GPT2Model
import torch
import torch.nn as nn
import torch.nn.functional as F


In [119]:
encoder_name = "bert-base-uncased"
decoder_name = "dbddv01/gpt2-french-small"

tok_en = AutoTokenizer.from_pretrained(encoder_name)
tok_fr = AutoTokenizer.from_pretrained(decoder_name)

encoder = AutoModel.from_pretrained(encoder_name)
decoder_backbone = GPT2Model.from_pretrained(decoder_name)


##  Détails mathématiques de la Cross-Attention

La **cross-attention** est le mécanisme qui permet au **décodeur** (GPT-2) de "regarder" les représentations produites par l’**encodeur** (BERT).  
Elle détermine quelles parties de la phrase source influencent la génération de chaque mot cible.

---

###  1. Les entrées

On dispose de trois matrices :

- $Q \in \mathbb{R}^{T_q \times d_k}$ : les **requêtes** (*queries*) venant du décodeur  
- $K \in \mathbb{R}^{T_k \times d_k}$ : les **clés** (*keys*) venant de l’encodeur  
- $V \in \mathbb{R}^{T_k \times d_v}$ : les **valeurs** (*values*) venant aussi de l’encodeur  

Chaque vecteur représente le sens d’un mot.  
La cross-attention apprend à relier les mots de la phrase cible (français) à ceux de la phrase source (anglais).

---

###  2. Calcul des scores d’attention

On mesure la similarité entre chaque requête et chaque clé :

$$
\text{scores} = \frac{QK^\top}{\sqrt{d_k}}
$$

- Le produit $QK^\top$ produit une matrice de taille $T_q \times T_k$, où chaque élément indique à quel point un mot cible “regarde” un mot source.  
- Le facteur $\frac{1}{\sqrt{d_k}}$ évite que les valeurs soient trop grandes quand $d_k$ augmente (stabilisation numérique).

---

###  3. Application du softmax

On transforme les scores en **poids de probabilité** :

$$
\text{poids} = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)
$$

Chaque ligne de cette matrice correspond à une distribution de probabilité :  
les poids indiquent l’importance relative de chaque mot anglais pour un mot français donné,  
et la somme des poids d’une ligne vaut $1$.

---

###  4. Combinaison avec les valeurs

Les poids sont utilisés pour combiner les représentations $V$ de l’encodeur :

$$
Z = \text{poids} \times V
$$

Chaque vecteur $Z_i$ est une **moyenne pondérée** des valeurs $V$,  
où les poids déterminent quels mots sources contribuent le plus à la prédiction du mot cible.  

Intuitivement :
- le modèle "pose une question" via $Q$,  
- et "cherche la réponse" dans la phrase source via $K$ et $V$.

---

###  5. Multi-head attention

En pratique, on ne fait pas une seule attention, mais plusieurs têtes en parallèle :

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \ldots, \text{head}_h)W^O
$$

Chaque tête apprend une manière différente de relier les mots  
(structure grammaticale, dépendances, contexte, etc.).  
Leur combinaison donne une vision plus complète des relations entre mots anglais et français.

---

###  En résumé

1. L’encodeur (BERT) produit des représentations $K$ et $V$ de la phrase anglaise.  
2. Le décodeur (GPT-2) génère ses requêtes $Q$ pour chaque mot français.  
3. La cross-attention calcule comment chaque $Q_i$ doit combiner les $V_j$ de la phrase source.  
4. Le résultat $Z$ guide la génération du mot suivant.

Ainsi, la couche de **cross-attention** fait le lien mathématique entre  
**la compréhension (encodeur)** et **la génération (décodeur)**.


In [39]:
class CrossAttention(nn.Module):
    def __init__(self, d_model=768, n_heads=12, dropout=0.1):
        super().__init__()
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.dropout = nn.Dropout(dropout)

    def _split_heads(self, x):
        B, T, D = x.shape
        return x.view(B, T, self.n_heads, self.d_head).transpose(1, 2)

    def _merge_heads(self, x):
        B, h, T, d = x.shape
        return x.transpose(1, 2).contiguous().view(B, T, h * d)

    def forward(self, q, k, v, key_padding_mask=None):
        Q = self._split_heads(self.q_proj(q))
        K = self._split_heads(self.k_proj(k))
        V = self._split_heads(self.v_proj(v))

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.d_head ** 0.5)
        if key_padding_mask is not None:
            mask = key_padding_mask.unsqueeze(1).unsqueeze(2)
            scores = scores.masked_fill(mask == 0, float('-inf'))

        attn = torch.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        context = torch.matmul(attn, V)
        context = self._merge_heads(context)
        return self.out_proj(context)


## Classe `EncoderDecoderNMT` — Architecture complète du traducteur

Cette classe assemble toutes les composantes du modèle de traduction :  
un **encodeur BERT**, un **décodeur GPT-2**, et une **couche de cross-attention** qui relie les deux.


### Fonctionnement du `forward`

1. **Encodage**  
   La phrase source (anglais) passe dans BERT :
   $$
   H_{enc} = \text{Encoder}(x_{en})
   $$

2. **Décodage partiel**  
   La phrase cible (français, décalée d’un mot) passe dans GPT-2 :
   $$
   H_{dec} = \text{Decoder}(y_{fr})
   $$

3. **Cross-Attention**  
   Le décodeur reçoit le contexte de l’encodeur :
   $$
   \text{context} = \text{CrossAttn}(H_{dec}, H_{enc}, H_{enc})
   $$

4. **Fusion et prédiction**  
   On combine le contexte et la sortie du décodeur :
   $$
   H_{fused} = \text{LayerNorm}(H_{dec} + \text{context})
   $$
   puis on prédit le mot suivant :
   $$
   \text{logits} = \text{Linear}(H_{fused})
   $$

5. **Calcul de la perte (optionnel)**  
   Si les étiquettes sont fournies :
   $$
   \text{loss} = \text{CrossEntropy}(\text{logits}, \text{labels})
   $$



In [40]:
class EncoderDecoderNMT(nn.Module):
    def __init__(self, encoder, decoder_backbone, tok_fr, tie_weights=True, n_heads=12, dropout=0.1):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder_backbone
        self.cross_attn = CrossAttention(d_model=self.decoder.config.n_embd,
                                         n_heads=n_heads, dropout=dropout)
        self.ln_fuse = nn.LayerNorm(self.decoder.config.n_embd)
        self.lm_head = nn.Linear(self.decoder.config.n_embd, self.decoder.config.vocab_size, bias=False)
        if tie_weights:
            self.lm_head.weight = self.decoder.wte.weight

    def forward(self, src_input_ids, src_attn_mask, tgt_input_ids, tgt_attn_mask=None, labels=None):
        enc_out = self.encoder(input_ids=src_input_ids, attention_mask=src_attn_mask, return_dict=True)
        H_enc = enc_out.last_hidden_state

        dec_out = self.decoder(input_ids=tgt_input_ids, attention_mask=tgt_attn_mask,
                               use_cache=False, return_dict=True)
        H_dec = dec_out.last_hidden_state

        context = self.cross_attn(H_dec, H_enc, H_enc, key_padding_mask=src_attn_mask)
        H_fused = self.ln_fuse(H_dec + context)
        logits = self.lm_head(H_fused)

        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)),
                                   labels.view(-1),
                                   ignore_index=-100)
        return {"logits": logits, "loss": loss}


In [41]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, df, tok_en, tok_fr, max_len=64):
        self.df = df.reset_index(drop=True)
        self.tok_en = tok_en
        self.tok_fr = tok_fr
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        en_text = self.df.loc[idx, "en"]
        fr_text = self.df.loc[idx, "fr"]

        src = self.tok_en(en_text, truncation=True, padding="max_length",
                          max_length=self.max_len, return_tensors="pt")
        tgt = self.tok_fr(fr_text, truncation=True, padding="max_length",
                          max_length=self.max_len, return_tensors="pt")

        bos_id = self.tok_fr.bos_token_id or self.tok_fr.eos_token_id
        pad_id = self.tok_fr.pad_token_id or self.tok_fr.eos_token_id

        tgt_in = torch.cat([torch.tensor([[bos_id]]),
                            tgt["input_ids"][:, :-1]], dim=1)
        labels = tgt["input_ids"].clone()
        labels[labels == pad_id] = -100

        return {
            "src_input_ids": src["input_ids"].squeeze(0),
            "src_attn_mask": src["attention_mask"].squeeze(0),
            "tgt_input_ids": tgt_in.squeeze(0),
            "tgt_attn_mask": (tgt_in != pad_id).long().squeeze(0),
            "labels": labels.squeeze(0),
        }


In [42]:
from torch.utils.data import DataLoader

train_data = TranslationDataset(train_df, tok_en, tok_fr, max_len=64)
val_data = TranslationDataset(val_df, tok_en, tok_fr, max_len=64)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)


In [43]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = EncoderDecoderNMT(encoder, decoder_backbone, tok_fr).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)


In [45]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()
device = "cuda"
model.to(device)

# Geler l'encodeur au début
for param in model.encoder.parameters():
    param.requires_grad = False

# Optimiseur
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)


/tmp/ipython-input-1973884016.py:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # pour le mixed precision


## 🔁 Boucle d’entraînement

Cette boucle entraîne le modèle sur plusieurs époques.  
Pour chaque lot de données, elle calcule la **perte (loss)**, effectue la **rétropropagation** et met à jour les poids du modèle.  
L’utilisation de `autocast()` permet un entraînement plus rapide et plus léger en **précision mixte (FP16)** sur GPU.


In [46]:
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        src_input_ids = batch["src_input_ids"].to(device)
        src_attn_mask = batch["src_attn_mask"].to(device)
        tgt_input_ids = batch["tgt_input_ids"].to(device)
        tgt_attn_mask = batch["tgt_attn_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        # 🔹 FP16 automatique
        with autocast():
            out = model(src_input_ids, src_attn_mask, tgt_input_ids, tgt_attn_mask, labels)
            loss = out["loss"]

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")


  0%|          | 0/19757 [00:00<?, ?it/s]/tmp/ipython-input-3434832626.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 19757/19757 [43:56<00:00,  7.49it/s]


Epoch 1 | Loss: 2.9251


100%|██████████| 19757/19757 [43:46<00:00,  7.52it/s]


Epoch 2 | Loss: 2.3738


100%|██████████| 19757/19757 [43:40<00:00,  7.54it/s]

Epoch 3 | Loss: 2.1341


In [48]:
torch.save(model.state_dict(), "bert2gpt2_translator.pt")


In [49]:
model.load_state_dict(torch.load("bert2gpt2_translator.pt", map_location=device))
model.eval()


EncoderDecoderNMT(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [50]:
def translate_sentence(model, sentence_en, tok_en, tok_fr, device, max_len=40):
    model.eval()
    with torch.no_grad():
        src = tok_en(sentence_en, return_tensors="pt", truncation=True, padding=True).to(device)
        enc_out = model.encoder(**src, return_dict=True).last_hidden_state

        bos_id = tok_fr.bos_token_id or tok_fr.eos_token_id
        generated = torch.tensor([[bos_id]], device=device)

        for _ in range(max_len):
            out = model(src["input_ids"], src["attention_mask"], generated)
            next_token = out["logits"][:, -1, :].argmax(-1, keepdim=True)
            generated = torch.cat([generated, next_token], dim=1)
            if next_token.item() == tok_fr.eos_token_id:
                break

        return tok_fr.decode(generated[0], skip_special_tokens=True)


 Les résultats montrent que le modèle traduit correctement des phrases simples et garde une bonne cohérence linguistique — un très bon signe que l’architecture **BERT→GPT2** apprend efficacement la correspondance entre l’anglais et le français.


In [84]:
sentence = "I love you "
print(translate_sentence_simple(model, sentence, tok_en, tok_fr, device,max_len=4))


Je vous aime.


In [93]:
sentence = "he is strong "
print(translate_sentence_simple(model, sentence, tok_en, tok_fr, device,max_len=4))

Il est fort.


In [117]:
sentence =" I feel sick "
print(translate_sentence_simple(model, sentence, tok_en, tok_fr, device,max_len=4))

Je me sens mal
